In [8]:
import mysql.connector as connection
import pandas as pd

In [3]:
try:
    mydb = connection.connect(host="localhost",user="root",passwd="Arvind@9799",use_pure = True)
    
    query =  "SELECT * FROM w3schools.employees"
    
    cursor = mydb.cursor()
    cursor.execute(query)
    print(cursor.fetchall())
    
except Exception as e:
    mydb.close()
    print(str(e))

[(1, 'Davolio', 'Nancy', datetime.date(1968, 12, 8), 'EmpID1.pic', "Education includes a BA in psychology from Colorado State University. She also completed (The Art of the Cold Call). Nancy is a member of 'Toastmasters International'."), (2, 'Fuller', 'Andrew', datetime.date(1952, 2, 19), 'EmpID2.pic', 'Andrew received his BTS commercial and a Ph.D. in international marketing from the University of Dallas. He is fluent in French and Italian and reads German. He joined the company as a sales representative, was promoted to sales manager and was then named vice president of sales. Andrew is a member of the Sales Management Roundtable, the Seattle Chamber of Commerce, and the Pacific Rim Importers Association.'), (3, 'Leverling', 'Janet', datetime.date(1963, 8, 30), 'EmpID3.pic', 'Janet has a BS degree in chemistry from Boston College). She has also completed a certificate program in food retailing management. Janet was hired as a sales associate and was promoted to sales representative.

In [9]:
query = "SELECT * FROM w3schools.employees"
df = pd.read_sql(query, mydb)
    
    # Save data to CSV file
csv_file = 'employees_data.csv'
df.to_csv(csv_file, index=False)
    
print(f"Data successfully exported to {csv_file}")
    

Data successfully exported to employees_data.csv


C:\Users\Dell\AppData\Local\Temp\ipykernel_40840\4240109480.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mydb)


In [10]:
pip install pandas pyarrow


Note: you may need to restart the kernel to use updated packages.


In [12]:
import pyarrow as pa
import pyarrow.parquet as pq

parquet_file = 'employees_data.parquet'
    
# Convert DataFrame to Parquet
table = pa.Table.from_pandas(df)
pq.write_table(table, parquet_file)
    
print(f"Data successfully exported to {parquet_file}")



Data successfully exported to employees_data.parquet


In [13]:
pip install mysql-connector-python pandas fastavro


In [24]:
import fastavro
from datetime import datetime,date
from io import BytesIO

In [19]:
avro_schema = {
    "type": "record",
    "name": "Employee",
    "fields": [
        {"name": "EmployeeID", "type": ["null", "int"]},
        {"name": "LastName", "type": ["null", "string"]},
        {"name": "FirstName", "type": ["null", "string"]},
        {"name": "BirthDate", "type": ["null", {"type": "int", "logicalType": "date"}]},
        {"name": "Photo", "type": ["null", "string"]},
        {"name": "Notes", "type": ["null", "string"]}
    ]
}

In [25]:
# Execute SQL query to fetch data
query = "SELECT * FROM w3schools.employees"
cursor = mydb.cursor(dictionary=True)  # Use dictionary cursor to fetch rows as dictionaries
cursor.execute(query)
rows = cursor.fetchall()
    
    # Convert date format to Avro logicalType
for row in rows:
    if row['BirthDate'] is not None:
# Convert datetime.date to Avro's logical type for date (milliseconds since Unix epoch)
        row['BirthDate'] = date.toordinal(row['BirthDate'])
    
    # Serialize data to Avro
with BytesIO() as avro_output:
    fastavro.writer(avro_output, avro_schema, rows)
        
        # Write Avro file
    with open('employees_data.avro', 'wb') as f:
        f.write(avro_output.getvalue())
    
print("Data successfully exported to employees_data.avro")

Data successfully exported to employees_data.avro
